In [3]:
#1a
import pandas as pd
import numpy as np
df = pd.read_csv('play.csv')
print(df.head())
total_samples = len(df)
play_yes = len(df[df['play'] == 'Yes'])
play_no = len(df[df['play'] == 'No'])

P_yes = play_yes / total_samples
P_no = play_no / total_samples
print(f"Prior Probability of 'Yes': {P_yes}")
print(f"Prior Probability of 'No': {P_no}")
def calculate_likelihood(feature, feature_value, target_value):
    subset = df[df['play'] == target_value]
    feature_count = len(subset[subset[feature] == feature_value])
    total_count = len(subset)
    return feature_count / total_count
test_sample = ['Rain', 'Cool', 'High', 'Strong']

likelihood_yes = 1
likelihood_no = 1

features = ['outlook', 'temp', 'humidity', 'wind']
for feature, value in zip(features, test_sample):
    likelihood_yes *= calculate_likelihood(feature, value, 'Yes')
    likelihood_no *= calculate_likelihood(feature, value, 'No')

print(f"Likelihood for 'Yes': {likelihood_yes}")
print(f"Likelihood for 'No': {likelihood_no}")
P_yes_given_x = P_yes * likelihood_yes
P_no_given_x = P_no * likelihood_no

print(f"Class conditional probability for 'Yes': {P_yes_given_x}")
print(f"Class conditional probability for 'No': {P_no_given_x}")
if P_yes_given_x > P_no_given_x:
    print("Predicted Class: Yes")
else:
    print("Predicted Class: No")


  day   outlook  temp humidity    wind play
0  D1     Sunny   Hot     High    Weak   No
1  D2     Sunny   Hot     High  Strong   No
2  D3  Overcast   Hot     High    Weak  Yes
3  D4      Rain  Mild     High    Weak  Yes
4  D5      Rain  Cool   Normal    Weak  Yes
Prior Probability of 'Yes': 0.6428571428571429
Prior Probability of 'No': 0.35714285714285715
Likelihood for 'Yes': 0.012345679012345678
Likelihood for 'No': 0.03840000000000001
Class conditional probability for 'Yes': 0.007936507936507936
Class conditional probability for 'No': 0.013714285714285719
Predicted Class: No


In [4]:
#1b
import pandas as pd
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
df = pd.read_csv('play.csv')
encoders={}
for i in df.columns:
    encoder = LabelEncoder()
    if df[i].dtype=='object':
        df[i]=encoder.fit_transform(df[i])
    encoders[i]=encoder
X = df[['outlook', 'temp', 'humidity', 'wind']]
y = df['play']
model = CategoricalNB()
model.fit(X, y)
test_sample = ['Rain', 'Cool', 'High', 'Strong']
encoded_test_sample = [[encoders[col].transform([val])[0] for col, val in zip(X.columns, test_sample)]]

model=CategoricalNB()
model.fit(X,y)
pred=model.predict(encoded_test_sample)
if (pred==1):
    print('yes')
else:
    print('no')

no


c:\Users\devgo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(


In [5]:
#2a
from collections import Counter
def entropy(y):
    total = len(y)
    counts = Counter(y)
    return -sum((count / total) * np.log2(count / total) for count in counts.values())
target_entropy=entropy(df['play'])
infogains={}
for feature in df.columns[:-1]:
    feature_entropy=0
    for value in np.unique(df[feature]):
        subset = df[df[feature] == value]['play']
        subset_entropy = entropy(subset)
        weight = len(subset) / len(df)
        feature_entropy += weight * subset_entropy
    infogains[feature] = target_entropy - feature_entropy

for feature,ig in infogains.items():
    print(f"{feature}:{ig}")

day:0.9402859586706311
outlook:0.24674981977443933
temp:0.02922256565895487
humidity:0.15183550136234159
wind:0.04812703040826949


In [6]:
#2b
rootnode=max(infogains,key=infogains.get)
print(rootnode)

day


In [7]:
#2c
#(a)
from sklearn.tree import DecisionTreeClassifier
from graphviz import Source
from sklearn import tree
model=DecisionTreeClassifier(criterion="entropy",max_depth=2,random_state=42)
model.fit(X,y)
tst=['Rain','Cool','High','Weak']
testenc= [[encoders[col].transform([val])[0] for col, val in zip(X.columns, tst)]]
#(b)
pred=model.predict(testenc)
decpred=encoders['play'].inverse_transform(pred)
print(decpred)
#(c)
graph = Source(tree.export_graphviz(model, out_file = None))
print(graph)

['No']
digraph Tree {
node [shape=box, fontname="helvetica"] ;
edge [fontname="helvetica"] ;
0 [label="x[0] <= 0.5\nentropy = 0.94\nsamples = 14\nvalue = [5, 9]"] ;
1 [label="entropy = 0.0\nsamples = 4\nvalue = [0, 4]"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="x[2] <= 0.5\nentropy = 1.0\nsamples = 10\nvalue = [5, 5]"] ;
0 -> 2 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;
3 [label="entropy = 0.722\nsamples = 5\nvalue = [4, 1]"] ;
2 -> 3 ;
4 [label="entropy = 0.722\nsamples = 5\nvalue = [1, 4]"] ;
2 -> 4 ;
}



c:\Users\devgo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [8]:
#2d
rootind=model.tree_.feature[0]
root=X.columns[rootind]
print(root)

outlook


In [9]:
#3a
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
diabetes = pd.read_csv("diabetes.csv")
X_diabetes = diabetes.iloc[:, :-1]
y_diabetes = diabetes.iloc[:, -1]
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_diabetes, y_diabetes, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_d = scaler.fit_transform(X_train_d)
X_test_d = scaler.transform(X_test_d)

nb_diabetes = GaussianNB()
nb_diabetes.fit(X_train_d, y_train_d)
y_pred_nb_d = nb_diabetes.predict(X_test_d)
print(accuracy_score(y_test_d,y_pred_nb_d))

dt_diabetes = DecisionTreeClassifier(random_state=42)
dt_diabetes.fit(X_train_d, y_train_d)
y_pred_dt_d = dt_diabetes.predict(X_test_d)
print(accuracy_score(y_test_d,y_pred_dt_d))

0.7662337662337663
0.7467532467532467


In [10]:
#3b
from sklearn.datasets import load_iris
i=load_iris()
iris=pd.DataFrame(i.data, columns=i.feature_names)
X_iris = iris.iloc[:, :-1]
y_iris = iris.iloc[:, -1]

label_encoder = LabelEncoder()
y_iris = label_encoder.fit_transform(y_iris)

X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(X_iris, y_iris, test_size=0.2, random_state=42)
nb_iris = GaussianNB()
nb_iris.fit(X_train_i, y_train_i)
y_pred_nb_i = nb_iris.predict(X_test_i)
print(accuracy_score(y_test_i,y_pred_nb_i))
dt_iris = DecisionTreeClassifier(random_state=42)
dt_iris.fit(X_train_i, y_train_i)
y_pred_dt_i = dt_iris.predict(X_test_i)
print(accuracy_score(y_test_i,y_pred_dt_i))

0.2
0.2
